In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
import sklearn
import warnings
import sksurv.datasets
import numpy as np
import joblib
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier

from itertools import product
from tqdm import tqdm
from xgbse import XGBSEKaplanNeighbors
from xgbse.converters import convert_to_structured
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.exceptions import UndefinedMetricWarning
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ParameterGrid
from sksurv.datasets import load_breast_cancer
from sksurv.metrics import cumulative_dynamic_auc
from sksurv.metrics import concordance_index_censored
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from sksurv.util import Surv


from sksurv.ensemble import GradientBoostingSurvivalAnalysis


warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
set_config(display="text")

In [2]:
# df_meteo=pd.read_csv('https://projet-incendie.s3.eu-west-3.amazonaws.com/Param%C3%A8tres_API.csv', sep=';')
# pd.set_option('display.max_rows', None)
# display(df_meteo)

In [3]:
df_corse=pd.read_csv('https://projet-incendie.s3.eu-west-3.amazonaws.com/dataset_modele_decompte.csv', sep=';')

pd.set_option('display.max_columns', None)
df_corse.head(10)

,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,TN50,TAMPLI,TNTXM,FFM,FXI,DXI,HXI,FXY,DXY,HXY,FXI3S,HXI3S,UN,HUN,UX,HUX,DHUMI40,DHUMI80,TSVM,UM,ORAG,BRUME,ETPMON,ETPGRILLE,Code INSEE_x,Code Postal,moyenne precipitations année,moyenne precipitations mois,moyenne evapotranspiration année,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,moyenne temperature mois,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),Surfaces agricoles (m2),Autres surfaces (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2),Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Type de peuplement,Nature,Feux,geo_point_2d,ville,Date,décompte,latitude,longitude,évènement,Feu prévu,Année,Mois,Nombre de feu par an,Nombre de feu par mois
0,0.0,NaN,10.8,NaN,22.2,NaN,NaN,16.50,NaN,NaN,11.4,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.80,0.56,NaN,NaN,NaN,NaN,11.43,10.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-25,19.0,41.926399,8.737603,False,1,2014,4,18,0
1,0.5,NaN,11.9,NaN,20.2,NaN,NaN,16.05,NaN,NaN,8.3,16.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.80,0.55,NaN,NaN,NaN,NaN,11.41,10.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-26,18.0,41.926399,8.737603,False,1,2014,4,18,0
2,8.2,NaN,10.9,NaN,19.0,NaN,NaN,14.95,NaN,NaN,8.1,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.80,0.55,NaN,NaN,NaN,NaN,11.41,10.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-27,17.0,41.926399,8.737603,False,1,2014,4,18,0
3,0.0,NaN,11.3,NaN,17.2,NaN,NaN,14.25,NaN,NaN,5.9,14.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.55,NaN,NaN,NaN,NaN,11.40,10.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-28,16.0,41.926399,8.737603,False,1,2014,4,18,0
4,0.4,NaN,13.1,NaN,20.6,NaN,NaN,16.85,NaN,NaN,7.5,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.55,NaN,NaN,NaN,NaN,11.40,10.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-29,15.0,41.926399,8.737603,False,1,2014,4,18,0
5,0.2,NaN,9.9,NaN,18.6,NaN,NaN,14.25,NaN,NaN,8.7,14.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.56,NaN,NaN,NaN,NaN,11.38,10.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-30,14.0,41.926399,8.737603,False,1,2014,4,18,0
6,0.2,NaN,9.8,NaN,21.2,NaN,NaN,15.50,NaN,NaN,11.4,15.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.55,NaN,NaN,NaN,NaN,11.37,10.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-05-01,13.0,41.926399,8.737603,False,1,2014,5,18,2
7,0.0,NaN,11.1,NaN,18.5,NaN,NaN,14.80,NaN,NaN,7.4,14.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.52,NaN,NaN,NaN,NaN,11.36,10.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-05-02,12.0,41.926399,8.737603,False,1,2014,5,18,2
8,0.0,NaN,9.1,NaN,19.8,NaN,NaN,14.45,NaN,NaN,10.7,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.55,NaN,NaN,NaN,NaN,11.35,10.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-05-03,11.0,41.926399,8.737603,False,1,2014,5,18,2
9,3.4,NaN,10.8,NaN,20.9,NaN,NaN,15.85,NaN,NaN,10.1,15.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,1.32,NaN,NaN,NaN,NaN,11.35,10.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-05-04,10.0,41.926399,8.737603,False,1,2014,5,18,2


In [4]:
df_corse['Feux'].value_counts()

Feux
0    268294
1      5251
Name: count, dtype: int64

In [5]:
df_corse.shape

(273545, 68)

In [6]:
pd.set_option('display.max_rows', None)
df_corse.isnull().sum()/len(df_corse)*100

RR                                             10.215504
DRR                                            93.894972
TN                                             16.173207
HTN                                            32.841032
TX                                             16.213055
HTX                                            32.832258
TM                                             30.399751
TMNX                                           16.408635
TNSOL                                          90.287521
TN50                                           90.276189
TAMPLI                                         16.408635
TNTXM                                          16.408635
FFM                                            47.274489
FXI                                            48.657077
DXI                                            48.730922
HXI                                            49.401013
FXY                                            52.537243
DXY                            

In [7]:
df_corse['Feu prévu'] = df_corse['Feu prévu'].astype(bool)
df_corse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273545 entries, 0 to 273544
Data columns (total 68 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   RR                                          245601 non-null  float64
 1   DRR                                         16700 non-null   float64
 2   TN                                          229304 non-null  float64
 3   HTN                                         183710 non-null  float64
 4   TX                                          229195 non-null  float64
 5   HTX                                         183734 non-null  float64
 6   TM                                          190388 non-null  float64
 7   TMNX                                        228660 non-null  float64
 8   TNSOL                                       26568 non-null   float64
 9   TN50                                        26599 non-null   float64
 

In [8]:
df_corse.head()

,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,TN50,TAMPLI,TNTXM,FFM,FXI,DXI,HXI,FXY,DXY,HXY,FXI3S,HXI3S,UN,HUN,UX,HUX,DHUMI40,DHUMI80,TSVM,UM,ORAG,BRUME,ETPMON,ETPGRILLE,Code INSEE_x,Code Postal,moyenne precipitations année,moyenne precipitations mois,moyenne evapotranspiration année,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,moyenne temperature mois,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),Surfaces agricoles (m2),Autres surfaces (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2),Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Type de peuplement,Nature,Feux,geo_point_2d,ville,Date,décompte,latitude,longitude,évènement,Feu prévu,Année,Mois,Nombre de feu par an,Nombre de feu par mois
0,0.0,NaN,10.8,NaN,22.2,NaN,NaN,16.50,NaN,NaN,11.4,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.80,0.56,NaN,NaN,NaN,NaN,11.43,10.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-25,19.0,41.926399,8.737603,False,True,2014,4,18,0
1,0.5,NaN,11.9,NaN,20.2,NaN,NaN,16.05,NaN,NaN,8.3,16.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.80,0.55,NaN,NaN,NaN,NaN,11.41,10.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-26,18.0,41.926399,8.737603,False,True,2014,4,18,0
2,8.2,NaN,10.9,NaN,19.0,NaN,NaN,14.95,NaN,NaN,8.1,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.80,0.55,NaN,NaN,NaN,NaN,11.41,10.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-27,17.0,41.926399,8.737603,False,True,2014,4,18,0
3,0.0,NaN,11.3,NaN,17.2,NaN,NaN,14.25,NaN,NaN,5.9,14.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.55,NaN,NaN,NaN,NaN,11.40,10.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-28,16.0,41.926399,8.737603,False,True,2014,4,18,0
4,0.4,NaN,13.1,NaN,20.6,NaN,NaN,16.85,NaN,NaN,7.5,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20004.0,20000,2.82,0.55,NaN,NaN,NaN,NaN,11.40,10.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Ajaccio,2014-04-29,15.0,41.926399,8.737603,False,True,2014,4,18,0


In [9]:
df_corse.columns

Index(['RR', 'DRR', 'TN', 'HTN', 'TX', 'HTX', 'TM', 'TMNX', 'TNSOL', 'TN50',
       'TAMPLI', 'TNTXM', 'FFM', 'FXI', 'DXI', 'HXI', 'FXY', 'DXY', 'HXY',
       'FXI3S', 'HXI3S', 'UN', 'HUN', 'UX', 'HUX', 'DHUMI40', 'DHUMI80',
       'TSVM', 'UM', 'ORAG', 'BRUME', 'ETPMON', 'ETPGRILLE', 'Code INSEE_x',
       'Code Postal', 'moyenne precipitations année',
       'moyenne precipitations mois', 'moyenne evapotranspiration année',
       'moyenne evapotranspiration mois', 'moyenne vitesse vent année',
       'moyenne vitesse vent mois', 'moyenne temperature année',
       'moyenne temperature mois', 'Surface parcourue (m2)',
       'Surface forêt (m2)', 'Surface maquis garrigues (m2)',
       'Autres surfaces naturelles hors forêt (m2)', 'Surfaces agricoles (m2)',
       'Autres surfaces (m2)', 'Surface autres terres boisées (m2)',
       'Surfaces non boisées naturelles (m2)',
       'Surfaces non boisées artificialisées (m2)',
       'Surfaces non boisées (m2)', 'Type de peuplement', 'Nat

In [10]:
df_variables = df_corse[[
    'moyenne precipitations mois', 'moyenne temperature mois',
    'moyenne evapotranspiration mois', 'moyenne vitesse vent année',
    'moyenne vitesse vent mois', 'moyenne temperature année',
    'RR', 'UM', 'ETPMON', 'TN', 'TX', 'évènement', 'décompte', 'Nombre de feu par an',
    'Nombre de feu par mois'
]]
df_variables.head()

,moyenne precipitations mois,moyenne temperature mois,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,RR,UM,ETPMON,TN,TX,évènement,décompte,Nombre de feu par an,Nombre de feu par mois
0,0.56,10.19,NaN,NaN,NaN,11.43,0.0,NaN,NaN,10.8,22.2,False,19.0,18,0
1,0.55,10.24,NaN,NaN,NaN,11.41,0.5,NaN,NaN,11.9,20.2,False,18.0,18,0
2,0.55,10.38,NaN,NaN,NaN,11.41,8.2,NaN,NaN,10.9,19.0,False,17.0,18,0
3,0.55,10.32,NaN,NaN,NaN,11.40,0.0,NaN,NaN,11.3,17.2,False,16.0,18,0
4,0.55,10.26,NaN,NaN,NaN,11.40,0.4,NaN,NaN,13.1,20.6,False,15.0,18,0


In [11]:
df_variables.rename(columns={
    'RR': 'Précipitations en mm',
    'TN': 'Température minimale sous abri',
    'TX': 'Température maximale sous abri',
    'UM': 'Humidité moyenne en %',
    'ETPMON': 'Evapotranspiration en mm',
       
}, inplace=True)
df_variables.head()

C:\Users\joelt\AppData\Local\Temp\ipykernel_28340\3994305555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_variables.rename(columns={


,moyenne precipitations mois,moyenne temperature mois,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,Précipitations en mm,Humidité moyenne en %,Evapotranspiration en mm,Température minimale sous abri,Température maximale sous abri,évènement,décompte,Nombre de feu par an,Nombre de feu par mois
0,0.56,10.19,NaN,NaN,NaN,11.43,0.0,NaN,NaN,10.8,22.2,False,19.0,18,0
1,0.55,10.24,NaN,NaN,NaN,11.41,0.5,NaN,NaN,11.9,20.2,False,18.0,18,0
2,0.55,10.38,NaN,NaN,NaN,11.41,8.2,NaN,NaN,10.9,19.0,False,17.0,18,0
3,0.55,10.32,NaN,NaN,NaN,11.40,0.0,NaN,NaN,11.3,17.2,False,16.0,18,0
4,0.55,10.26,NaN,NaN,NaN,11.40,0.4,NaN,NaN,13.1,20.6,False,15.0,18,0


In [12]:
print(df_variables.columns)

Index(['moyenne precipitations mois', 'moyenne temperature mois',
       'moyenne evapotranspiration mois', 'moyenne vitesse vent année',
       'moyenne vitesse vent mois', 'moyenne temperature année',
       'Précipitations en mm', 'Humidité moyenne en %',
       'Evapotranspiration en mm', 'Température minimale sous abri',
       'Température maximale sous abri', 'évènement', 'décompte',
       'Nombre de feu par an', 'Nombre de feu par mois'],
      dtype='object')


#### XGBoost Régression classique

In [ ]:

threshold = 0.5
cols_to_keep = df_corse.columns[df_corse.isnull().mean() < threshold]
df_clean = df_corse[cols_to_keep].copy()

features = [
    "RR", "TN", "TX", "TM", "UM", "ETPGRILLE", "ETPMON",
    "moyenne precipitations année", "moyenne precipitations mois",
    "moyenne evapotranspiration année", "moyenne evapotranspiration mois",
    "moyenne vitesse vent année", "moyenne vitesse vent mois",
    "moyenne temperature année", "moyenne temperature mois",
    "Nombre de feu par an", "Nombre de feu par mois"
]
features = [f for f in features if f in df_clean.columns]


df_clean = df_clean.dropna(subset=["décompte"])


X = df_clean[features]
y = df_clean["décompte"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("xgb", XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42))
])

pipeline.fit(X_train, y_train)

score = pipeline.score(X_test, y_test)
print(f"R² score sur test : {score:.3f}")

X_imputed = pipeline.named_steps["imputer"].transform(X)
print(f"Nombre de NaN après imputation: {np.isnan(X_imputed).sum()}")

R² score sur test : 0.459
Nombre de NaN après imputation: 0


#### XGBoost Regressor Survival Cox

In [14]:
df_clean = df_clean.rename(columns={"Feu prévu": "event", "décompte": "duration"})

y_structured = Surv.from_dataframe("event", "duration", df_clean)

X = df_clean[features]
y = y_structured

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

event_train = y_train["event"]
duration_train = y_train["duration"]

event_test = y_test["event"]
duration_test = y_test["duration"]

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("xgb", XGBRegressor(
        objective="survival:cox",
        n_estimators=100,
        learning_rate=0.1,
        max_depth=4,
        tree_method="hist",
        device="cuda",       
        random_state=42
    ))
])

pipeline.fit(X_train, duration_train, xgb__sample_weight=event_train)

risques = pipeline.predict(X_test)

cindex = concordance_index_censored(event_test, duration_test, risques)[0]
print(f"C-index test : {cindex:.3f}")


c:\Users\joelt\anaconda3\envs\pytorch_deep\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:13:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


C-index test : 0.768


#### GridSearch et XGBoost

In [15]:
param_grid = {
    "n_estimators": [50, 100],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5]
}

params_list = list(product(param_grid["n_estimators"],
                           param_grid["learning_rate"],
                           param_grid["max_depth"]))

best_cindex = -float('inf')
best_params = None
best_model = None
results = []

for n_estimators, learning_rate, max_depth in tqdm(params_list, desc="Grid search XGBoost GPU"):
    model = xgb.XGBRegressor(
        objective="survival:cox",
        tree_method="hist",
        device="cuda",
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    
    model.fit(X_train, y_train["duration"], sample_weight=y_train["event"])
    
    risques = model.predict(X_test)
    cindex = concordance_index_censored(y_test["event"], y_test["duration"], risques)[0]
    
    results.append({
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "cindex": cindex
    })
    
    if cindex > best_cindex:
        best_cindex = cindex
        best_params = {
            "n_estimators": n_estimators,
            "learning_rate": learning_rate,
            "max_depth": max_depth
        }
        best_model = model

print(f"Meilleur C-index GPU : {best_cindex:.3f}")
print("Meilleurs paramètres :", best_params)

xgb_results = pd.DataFrame(results)
print(xgb_results.sort_values("cindex", ascending=False))

Grid search XGBoost GPU: 100%|██████████| 8/8 [01:44<00:00, 13.01s/it]

Meilleur C-index GPU : 0.776
Meilleurs paramètres : {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5}
   n_estimators  learning_rate  max_depth    cindex
7           100           0.10          5  0.776043
5           100           0.05          5  0.769760
3            50           0.10          5  0.769755
1            50           0.05          5  0.764765
6           100           0.10          3  0.762848
2            50           0.10          3  0.758863
4           100           0.05          3  0.758742
0            50           0.05          3  0.757332


In [16]:
best_params

{'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5}

In [17]:
# # Sauvegarde du modèle
# xgb_results_sorted = xgb_results.sort_values("cindex", ascending=False)

# xgb_results_sorted.to_csv("xgb_results.csv", index=False)


In [18]:
xgb_score=pd.read_csv("xgb_results.csv")
xgb_score

,n_estimators,learning_rate,max_depth,cindex
0,100,0.10,5,0.776043
1,100,0.05,5,0.769760
2,50,0.10,5,0.769755
3,50,0.05,5,0.764765
4,100,0.10,3,0.762848
5,50,0.10,3,0.758863
6,100,0.05,3,0.758742
7,50,0.05,3,0.757332


In [19]:
# # Sauvegarder le modèle dans un fichier
# best_model.save_model("best_xgb_model.json")
# print("Modèle sauvegardé sous best_xgb_model.json")


In [20]:
# # Charger le modèle depuis le fichier
# loaded_model = xgb.XGBRegressor()
# loaded_model.load_model("best_xgb_model.json")
# print("Modèle chargé depuis best_xgb_model.json")

# # Tu peux maintenant faire des prédictions avec loaded_model
# # preds = loaded_model.predict(X_new)

In [21]:
# loaded_model = xgb.XGBRegressor()
# loaded_model.load_model("best_xgb_model.json")

# X_new = df_clean[features]

# predictions = loaded_model.predict(X_new)

# df_clean['predictions'] = predictions
# display(df_clean.head(100))